# **Interactive Visual Analytics with Folium**

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. Here, we will be performing more interactive visual analytics using `Folium`.

In [1]:
#import piplite
#await piplite.install(['folium'])
#await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
## Task 1: Mark all launch sites on a map
worldmap = folium.Map()

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
#from js import fetch
#import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
#resp = await fetch(URL)
#spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(URL)

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
launch_sites_df
for i in launch_sites_df.values:
    print (i)
    break

['CCAFS LC-40' 28.56230197 -80.57735648]


In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i in launch_sites_df.values:
    circle = folium.Circle([i[1], i[2]], radius=40, color='#d35400', fill=True).add_child(folium.Popup(i[0]))
    marker = folium.map.Marker([i[1], i[2]], 
                               icon=DivIcon(icon_size=(20,20),
                                            icon_anchor=(0,0), 
                                            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % i[0], 
                                           )
                              )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [12]:
spacex_df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [13]:
# Task 2: Mark the success/failed launches for each site on the map
launch_result = MarkerCluster().add_to(site_map)

for lat, lng, label, in zip(spacex_df['Lat'], spacex_df['Long'], spacex_df['class']):
    folium.Marker(location=[lat, lng],
                  icon=None,
                  popup=label,
                  ).add_to(launch_result)

site_map

In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [15]:
marker_cluster = MarkerCluster()

Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value

In [16]:
marker_color = []
for i in spacex_df['class'].values:
    if i == 1:
        marker_color.append("green")
    else:
        marker_color.append("red")

spacex_df['marker_color'] = marker_color
spacex_df.tail(5)
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [17]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    #print (record)
    #break
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(location=[record.iloc[1], record.iloc[2]],
                           icon=folium.Icon(color='white', icon_color=record.iloc[4]),
                           popup=record.iloc[0],
                           tooltip = record.iloc[0],
                          )
    marker_cluster.add_child(marker)

site_map

In [18]:
# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
launch_sites_coordinates = spacex_df.groupby('Launch Site')[['Lat', 'Long']].first().reset_index()
launch_sites_coordinates[launch_sites_coordinates['Launch Site'] == 'CCAFS SLC-40'].values

array([['CCAFS SLC-40', 28.56319718, -80.57682003]], dtype=object)

In [22]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat = launch_sites_coordinates[launch_sites_coordinates['Launch Site'] == 'CCAFS SLC-40'].values[0][1]
launch_site_lon = launch_sites_coordinates[launch_sites_coordinates['Launch Site'] == 'CCAFS SLC-40'].values[0][2]
coastline_lat = 28.56328
coastline_lon = -80.56785
railway_lat = 28.56379
railway_lon = -80.58688
highway_lat = 28.56306
highway_lon = -80.57073
city_titusville_lat = 28.6159
city_titusville_lon = -80.8147
city_melbourne_lat = 28.07933
city_melbourne_lon = -80.6051
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_titusville = calculate_distance(launch_site_lat, launch_site_lon, city_titusville_lat, city_titusville_lon)
#distance_melbourne = calculate_distance(launch_site_lat, launch_site_lon, city_melbourne_lat, city_melbourne_lon)

print (distance_coastline, "Km")
print (distance_railway, "Km")
print (distance_highway, "Km")
print (distance_titusville, "Km")
#print (distance_melbourne, "Km")

0.8763489675419277 Km
0.9849862408470367 Km
0.5951437226822706 Km
23.96131007631149 Km


In [23]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_coastline_marker = folium.Marker([coastline_lat, coastline_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
                                            ),
                               )

distance_railway_marker = folium.Marker([railway_lat, railway_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
                                            ),
                               )

distance_highway_marker = folium.Marker([highway_lat, highway_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
                                            ),
                               )

distance_titusville_marker = folium.Marker([city_titusville_lat, city_titusville_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_titusville),
                                            ),
                               )

#distance_melbourne_marker = folium.Marker([city_melbourne_lat, city_melbourne_lon],
#                                icon=DivIcon(icon_size=(20,20),
#                                             icon_anchor=(0,0),
#                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_melbourne),
#                                            ),
#                               )

marker_cluster.add_child(distance_coastline_marker)
marker_cluster.add_child(distance_railway_marker)
marker_cluster.add_child(distance_highway_marker)
marker_cluster.add_child(distance_titusville_marker)
#marker_cluster.add_child(distance_melbourne_marker)
site_map

Draw a `PolyLine` between a launch site to the selected coastline point


In [24]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
costline_line_coordinates = [[coastline_lat, coastline_lon], [launch_site_lat, launch_site_lon]]
railway_line_coordinates = [[railway_lat, railway_lon], [launch_site_lat, launch_site_lon]]
highway_line_coordinates = [[highway_lat, highway_lon], [launch_site_lat, launch_site_lon]]
titusville_line_coordinates = [[city_titusville_lat, city_titusville_lon], [launch_site_lat, launch_site_lon]]
#melbourne_line_coordinates = [[city_melbourne_lat, city_melbourne_lon], [launch_site_lat, launch_site_lon]]
lines=folium.PolyLine(locations=costline_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=railway_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=highway_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=titusville_line_coordinates, weight=1)
site_map.add_child(lines)
#lines=folium.PolyLine(locations=melbourne_line_coordinates, weight=1)
##site_map.add_child(lines)

In [25]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat = launch_sites_coordinates[launch_sites_coordinates['Launch Site'] == 'KSC LC-39A'].values[0][1]
launch_site_lon = launch_sites_coordinates[launch_sites_coordinates['Launch Site'] == 'KSC LC-39A'].values[0][2]
coastline_lat = 28.56825
coastline_lon = -80.60728
railway_lat = 28.57325
railway_lon = -80.65419
highway_lat = 28.57359
highway_lon = -80.65378
city_titusville_lat = 28.61259
city_titusville_lon = -80.8147
city_melbourne_lat = 28.07933
city_melbourne_lon = -80.6051
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_titusville = calculate_distance(launch_site_lat, launch_site_lon, city_titusville_lat, city_titusville_lon)

print (distance_coastline, "Km")
print (distance_railway, "Km")
print (distance_highway, "Km")
print (distance_titusville, "Km")

# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_coastline_marker = folium.Marker([coastline_lat, coastline_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
                                            ),
                               )

distance_railway_marker = folium.Marker([railway_lat, railway_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
                                            ),
                               )

distance_highway_marker = folium.Marker([highway_lat, highway_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
                                            ),
                               )

distance_titusville_marker = folium.Marker([city_titusville_lat, city_titusville_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_titusville),
                                            ),
                               )

marker_cluster.add_child(distance_coastline_marker)
marker_cluster.add_child(distance_railway_marker)
marker_cluster.add_child(distance_highway_marker)
marker_cluster.add_child(distance_titusville_marker)
site_map

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
costline_line_coordinates = [[coastline_lat, coastline_lon], [launch_site_lat, launch_site_lon]]
railway_line_coordinates = [[railway_lat, railway_lon], [launch_site_lat, launch_site_lon]]
highway_line_coordinates = [[highway_lat, highway_lon], [launch_site_lat, launch_site_lon]]
titusville_line_coordinates = [[city_titusville_lat, city_titusville_lon], [launch_site_lat, launch_site_lon]]
lines=folium.PolyLine(locations=costline_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=railway_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=highway_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=titusville_line_coordinates, weight=1)
site_map.add_child(lines)

3.9096536573987866 Km
0.7125674843468542 Km
0.6735504316000418 Km
16.96254178680626 Km


In [26]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat = launch_sites_coordinates[launch_sites_coordinates['Launch Site'] == 'VAFB SLC-4E'].values[0][1]
launch_site_lon = launch_sites_coordinates[launch_sites_coordinates['Launch Site'] == 'VAFB SLC-4E'].values[0][2]
coastline_lat = 34.63353
coastline_lon = -120.62607
railway_lat = 34.63367
railway_lon = -120.62472
highway_lat = 34.68269
highway_lon = -120.6038
city_lat = 34.63897
city_lon = -120.45791
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

print (distance_coastline, "Km")
print (distance_railway, "Km")
print (distance_highway, "Km")
print (distance_city, "Km")

# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_coastline_marker = folium.Marker([coastline_lat, coastline_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
                                            ),
                               )

distance_railway_marker = folium.Marker([railway_lat, railway_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
                                            ),
                               )

distance_highway_marker = folium.Marker([highway_lat, highway_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
                                            ),
                               )

distance_city_marker = folium.Marker([city_lat, city_lon],
                                icon=DivIcon(icon_size=(20,20),
                                             icon_anchor=(0,0),
                                             html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
                                            ),
                               )

marker_cluster.add_child(distance_coastline_marker)
marker_cluster.add_child(distance_railway_marker)
marker_cluster.add_child(distance_highway_marker)
marker_cluster.add_child(distance_city_marker)
site_map

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
costline_line_coordinates = [[coastline_lat, coastline_lon], [launch_site_lat, launch_site_lon]]
railway_line_coordinates = [[railway_lat, railway_lon], [launch_site_lat, launch_site_lon]]
highway_line_coordinates = [[highway_lat, highway_lon], [launch_site_lat, launch_site_lon]]
city_line_coordinates = [[city_lat, city_lon], [launch_site_lat, launch_site_lon]]
lines=folium.PolyLine(locations=costline_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=railway_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=highway_line_coordinates, weight=1)
site_map.add_child(lines)
lines=folium.PolyLine(locations=city_line_coordinates, weight=1)
site_map.add_child(lines)

1.4046433794201971 Km
1.2823299819862655 Km
5.581748050519044 Km
14.003796752895953 Km
